In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
import ShockFunction
from DB2Gams import OrdSet as OS
os.chdir(directory['curr'])

The file_gams_py_gdb1.gdx is still active and was not deleted.


*Specify settings:*

In [40]:
# catalog = "main_catalog"
# currapp = True

In [3]:
name = 'A3'
f = {'tech': directory['data']+"/techdata_" + catalog + ".xlsx",
     'other': directory['data']+"/Othdata_" + catalog + ".xlsx",
     'IOdata':directory['data'] + "/IOdata_" + catalog.replace("_ID", "").replace("_EOP","") + ".xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.
n_steps_factor = 0.30

In [4]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [5]:
%run ATree.ipynb

## 1: Initialize model

*Set up gams model from tree:*

In [6]:
if catalog in ["main_catalog", "categoryexperiment_EOP"]:
    use_EOP = True
elif catalog in ["categoryexperiment_ID"]:
    use_EOP = False

In [7]:
gm = abatement.abate(nts=nts,tech=tech,use_EOP=use_EOP,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': name})
gm.model.settings.catalog = catalog

*Add a couple of extra aliases, and update sets with elements:*

In [8]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)
# gm.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}


*Initialise variables at leontief levels, and fill out with simple standard values in potentially missing domains:*

In [9]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
    kwargs_mi = {'opt_file': file.read()}

In [10]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

## 2: Solve model


*Set up baseline model and solve:*

In [11]:
gm.write_and_run(name='baseline', kwargs_mi=kwargs_mi)
db0 = gm.model_instances['baseline'].out_db
print(gm.model_instances["baseline"].modelstat)

16.0



*Read back solution to main db:*

In [12]:
gm.model.database.merge_dbs(gm.model.database,db0,'second')

*Turn exogenous parameters into place*

In [13]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
eta_star = gm.get("eta").copy()
mu_star = gm.get("mu").copy()
if "EOP" in gm.state:
    sigmaG_star = gm.get("sigmaG").copy()

In [14]:
eta_star[:] = -10
# sigma_star[gm.get("kno_ID_EC")] = 0.2
sigma_star[gm.get("kno_ID_CU")] = 0.3 #0.3
sigma_star[gm.get("kno_ID_BX")] = 0.4
# sigma_star["Y"] = 0.5

mu_star[gm.get("map_ID_CU")] = 1.0

if "EOP" in gm.state:
    mu_star[gm.get("map_EOP_CU")] = 1.0
    sigma_star[gm.get("kno_EOP_CU")] = 0.3
    sigmaG_star[:] = 8
    db_star["sigmaG"] = sigmaG_star

    
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star
db_star["mu"] = mu_star

In [15]:
name='v0'
gm.write_and_run(name=name, add_checkpoint=name, kwargs_mi=kwargs_mi)
gm.checkpoints['v1'] = gm.model_instances[name].ws.add_checkpoint()
print(gm.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = gm.checkpoints[name], options_run={'checkpoint': gm.checkpoints['v1']}, kwargs_shock={"n_steps":int(100*n_steps_factor)}))
db1 = gm.model_instances[name].out_db
gm.model.database.merge_dbs(gm.model.database,db1,'second')

{'Modelstat': 16.0, 'Solvestat': 1.0}


*Change to calibration state and resolve:*

In [16]:
if catalog in ["categoryexperiment_EOP"]:
    state = "EOP_calibrate_CNS"
elif catalog in ["main_catalog"]:
    if currapp:
        state = "EOP_calibrate"
    else: 
        state = "EOP_calibrate_CNS"
elif catalog in ["categoryexperiment_ID"]:
    state = "ID_calibrate_CNS"

In [17]:
gm.reset_settings()
gm.setstate(state,init=False)

In [18]:
gm.write_and_run(name=state,kwargs_init={'check_variables':True},overwrite=True, kwargs_mi=kwargs_mi)
db0 = gm.model_instances[state].out_db

In [19]:
gm.model.database.merge_dbs(gm.model.database,db0,"second")

In [20]:
for sym in gm.model.database.symbols:
    if "_l1" in sym:
        del gm.model.database.series[sym]

### Make this work:

In [21]:
gm.model_instances[state].modelstat

2.0

In [22]:
if "CNS" in gm.state:
    db = gm.add_calib_data(f["IOdata"], currapp=False)
else:
    db = gm.add_calib_data(f["IOdata"], currapp=True)

In [23]:
if "EOP" in gm.state:
    db["sigma"] = (gm.get("sigma")[gm.get("kno_ID_CU")]*0 + 1.2).append(gm.get("sigma")[gm.get("kno_EOP_CU")]*0 + 1.2)
    if not "CNS" in gm.state:
        db["sigmaGbar"] = gm.get("sigmaGbar")[gm.get("EOP_out")]*0 + 0.01
else:
    db["sigma"] = gm.get("sigma")[gm.get("kno_ID_CU")]*0 + 1.2

In [24]:
gm.write_and_run(name=state+'IO',kwargs_init={'check_variables':True},overwrite=True,add_checkpoint=state+'IO', kwargs_mi=kwargs_mi)
gm.checkpoints[state+'IO' + "_cp"] = gm.model_instances[state+'IO'].ws.add_checkpoint()

In [25]:
vars_ = ["qD", "PwThat", "mu", "gamma_tau", "pMhat", "pM", "qS"]
if "EOP" in gm.state:
    vars_ += ["sigmaG", "muG"]
store_sol = {var:{"domains":"l1"} for var in vars_}

In [26]:
print(gm.model_instances[state+'IO'].solve_sneakily(db_star=db, from_cp = True, cp_init = gm.checkpoints[state+'IO'], options_run={'checkpoint': gm.checkpoints[state+'IO'+"_cp"]}, kwargs_shock={"n_steps":int(100*n_steps_factor), "store_sol":store_sol, 'solvetext': gm.model.settings.get_conf('solve')}))

{'Modelstat': 2.0, 'Solvestat': 1.0}


In [27]:
db1 = gm.model_instances[state+'IO'].out_db
gm.model.database.merge_dbs(gm.model.database,db1,"second")

In [28]:
for sym in gm.model.database.symbols:
    if "_l1" in sym:
        del gm.model.database.series[sym]

### Now turn sigma all the way to 10 and sigmaG towards 0.1

In [29]:
sigmaG_val = {"main_catalog":0.01, "categoryexperiment_ID":0.45, "categoryexperiment_EOP":0.45}
MNL_sigma = {"main_catalog":10, "categoryexperiment_ID":30, "categoryexperiment_EOP":30}

In [30]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
if "EOP" in gm.state:
    db_star["sigma"] = (gm.get("sigma")[gm.get("kno_ID_CU")]*0 + MNL_sigma[catalog]).append(gm.get("sigma")[gm.get("kno_EOP_CU")]*0 + MNL_sigma[catalog])
    if "CNS" in gm.state:
        db_star["sigmaG"] = gm.get("sigmaG")*0 + sigmaG_val[catalog]
elif "ID" in gm.state:
    db_star["sigma"] = gm.get("sigma")[gm.get("kno_ID_CU")]*0 + MNL_sigma[catalog]

In [31]:
gm.write_and_run(name=state+'_sigma',kwargs_init={'check_variables':True},overwrite=True,add_checkpoint=state+'_sigma', kwargs_mi=kwargs_mi)
gm.checkpoints[state+'_sigma' + "_cp"] = gm.model_instances[state+'_sigma'].ws.add_checkpoint()

In [32]:
print(gm.model_instances[state+'_sigma'].solve_sneakily(db_star=db_star, from_cp = True, cp_init = gm.checkpoints[state+'_sigma'], options_run={'checkpoint': gm.checkpoints[state+'_sigma'+"_cp"]}, kwargs_shock={"n_steps":int(100*n_steps_factor), "store_sol":store_sol, 'solvetext': gm.model.settings.get_conf('solve')}))

{'Modelstat': 2.0, 'Solvestat': 1.0}


In [33]:
gm.model.database.merge_dbs(gm.model.database,gm.model_instances[state+"_sigma"].out_db,"second")

In [34]:
for sym in gm.model.database.symbols:
    if "_l1" in sym:
        del gm.model.database.series[sym]

#### For figures, we move back the muG and sigmaG to make figures less smooth

In [35]:
if False:
    gm.reset_settings()
    gm.setstate("EOP",init=False)

    gm.write_and_run(name=state+'_G',kwargs_init={'check_variables':True},overwrite=True,add_checkpoint=state+'_G', kwargs_mi=kwargs_mi)
    gm.checkpoints[state+'_G' + "_cp"] = gm.model_instances[state+'_G'].ws.add_checkpoint()

    db_star = DataBase.GPM_database(**{'name': 'db_star'})
    db_star["muG"] = gm.get("muG")*0
    db_star["sigmaG"] = gm.get("sigmaG")*0 + sigmaG_val[catalog]

    print(gm.model_instances[state+'_G'].solve_sneakily(db_star=db_star, from_cp = True, cp_init = gm.checkpoints[state+'_G'], options_run={'checkpoint': gm.checkpoints[state+'_G'+"_cp"]}, kwargs_shock={"n_steps":250, "store_sol":store_sol, 'solvetext': gm.model.settings.get_conf('solve')}))

    gm.model.database.merge_dbs(gm.model.database,gm.model_instances[state+"_G"].out_db,"second")

In [36]:
for sym in gm.model.database.symbols:
    if "_l1" in sym:
        del gm.model.database.series[sym]
for sym in store_sol:
    del gm.model.database.series["sol_" + sym]

In [37]:
gm.export(name=catalog)

'C:\\Users\\zgr679\\Documents\\GitHub\\GPM_v05\\examples\\Abatement\\gamsmodels\\main_catalog'

In [38]:
if catalog == "categoryexperiment_EOP":
    print("Emission coefficient in ID should be " + str(round(((gm.get("unit_costs_EOP")["EOP_t1"] - ((gm.get("mu")[gm.get("map_ID_BX")] * gm.get("PwT")).loc[(["basetech_EH_K", "basetech_EH_electricity", "basetech_EH_gas"], ["basetech_EH"])]).sum())/gm.get("mu").loc[(["basetech_EH_oil"], ["basetech_EH"])].values[0] - gm.get("PwT")["oil"])/gm.get("unit_costs_EOP")["EOP_t1"], 2)))

In [39]:
# %run A3_CO2shock.ipynb